<a href="https://colab.research.google.com/github/MSISCapstone/CSPatchTST/blob/mouryavs/WPatchTST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone -b mouryavs https://github.com/MSISCapstone/CSPatchTST.git

Cloning into 'CSPatchTST'...
remote: Enumerating objects: 694, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 694 (delta 40), reused 25 (delta 11), pack-reused 610 (from 1)
Receiving objects: 100% (694/694), 42.15 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (349/349), done.
Filtering content: 100% (2/2), 221.27 MiB | 23.62 MiB/s, done.


In [2]:
cd /content/CSPatchTST/WPatchTST


/content/CSPatchTST/WPatchTST


In [3]:
ls

environment.yml  requirements.txt  WPatchTST_Environment.ipynb


In [4]:
%pip install -r requirements.txt



In [5]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


2.9.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB


In [6]:
%cd /content/CSPatchTST
!ls


/content/CSPatchTST
datasets				PatchTST_supervised
docs					pic
encoder_for_short_channel_inputs.ipynb	README.md
LICENSE					WPatchTST
notebooks				WPatchTST.ipynb
PatchTST_self_supervised


In [7]:
!grep -R "patch_len" -n . | head -n 25


./PatchTST_supervised/scripts/PatchTST/etth2.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/electricity.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/ettm1.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh:23:patch_lengths="6,12,24"      # Three temporal scales
./PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh:43:      --patch_lengths $patch_lengths \
./PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh:47:      --short_patch_len 6 \      #Short Patch Config VV
./PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh:58:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/etth1.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/ettm2.sh:38:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/univariate/etth2.sh:43:      --patch_len 16\
./PatchTST_supervised/scripts/PatchTST/univariate/ettm1.sh:43:     

In [8]:
!sed -n '1,120p' PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


#!/bin/bash

# Multi-Scale + Cross-Channel PatchTST for Weather Forecasting
# Combines both enhancements for maximum performance:
#   - Multi-scale: Captures temporal patterns at 6hr, 12hr, 24hr scales
#   - Cross-channel: Enables interaction between weather variables

model_name=PatchTST
random_seed=2021

root_path_name=./dataset/
data_path_name=weather.csv
model_id_name=weather
data_name=custom

seq_len=336

# Enable BOTH enhancements
multi_scale=1
channel_independent=0        # 0 = cross-channel (variables interact)

# Multi-scale configuration
patch_lengths="6,12,24"      # Three temporal scales
patch_strides="3,6,12"       # 50% overlap for each
patch_weights="0.2,0.5,0.3"  # Balanced fusion

for pred_len in 96 192 336 720
do
    python -u run_longExp.py \
      --random_seed $random_seed \
      --is_training 1 \
      --root_path $root_path_name \
      --data_path $data_path_name \
      --model_id $model_id_name'_multiCross_'$seq_len'_'$pred_len \
      --model $model_name \
 

In [9]:
!cp PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh \
    PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh.bak


In [10]:
!grep -R "python -u" -n PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


29:    python -u run_longExp.py \


In [11]:
!grep -R "argparse" -n PatchTST_supervised | head -n 20


PatchTST_supervised/Formers/Pyraformer/single_step_main.py:1:import argparse
PatchTST_supervised/Formers/Pyraformer/single_step_main.py:241:    parser = argparse.ArgumentParser()
PatchTST_supervised/Formers/Pyraformer/long_range_main.py:1:import argparse
PatchTST_supervised/Formers/Pyraformer/long_range_main.py:311:    parser = argparse.ArgumentParser()
PatchTST_supervised/Formers/Pyraformer/pyraformer/graph_attention.py:12:import argparse
PatchTST_supervised/Formers/Pyraformer/pyraformer/graph_attention.py:428:    parser = argparse.ArgumentParser(description='Needed for graph self attention.')
PatchTST_supervised/Formers/FEDformer/run.py:1:import argparse
PatchTST_supervised/Formers/FEDformer/run.py:13:parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')
PatchTST_supervised/run_longExp.py:1:import argparse
PatchTST_supervised/run_longExp.py:9:    parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Tim

In [12]:
!sed -n '1,200p' PatchTST_supervised/run_longExp.py


import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

    # random seed
    parser.add_argument('--random_seed', type=int, default=2021, help='random seed')

    # basic config
    parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
    parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
    parser.add_argument('--model', type=str, required=True, default='Autoformer',
                        help='model name, options: [Autoformer, Informer, Transformer]')

    # data loader
    parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
    parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
    parser.add_argument('--data_path', type=str,

In [13]:
!grep -R "unfold(" -n PatchTST_supervised | head -n 20


PatchTST_supervised/layers/PatchTST_backbone.py:78:        z = z.unfold(dimension=-1, size=self.patch_len, step=self.stride)                   # z: [bs x nvars x patch_num x patch_len]


In [14]:
!grep -R "PatchTST_backbone" -n PatchTST_supervised | head -n 20


PatchTST_supervised/models/PatchTST.py:7:from layers.PatchTST_backbone import PatchTST_backbone
PatchTST_supervised/models/PatchTST.py:67:            encoder = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:129:    - Run short group through single-scale PatchTST_backbone with short_patch_len/short_stride (model_short)
PatchTST_supervised/models/PatchTST.py:258:                    self.model_trend_short = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:282:                    self.model_res_short = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:324:                    self.model_short = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:345:                    self.model_trend = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:357:                    self.model_res = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:373:                    self.model_trend_long = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:385:       

In [15]:
!sed -n '1,120p' PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


#!/bin/bash

# Multi-Scale + Cross-Channel PatchTST for Weather Forecasting
# Combines both enhancements for maximum performance:
#   - Multi-scale: Captures temporal patterns at 6hr, 12hr, 24hr scales
#   - Cross-channel: Enables interaction between weather variables

model_name=PatchTST
random_seed=2021

root_path_name=./dataset/
data_path_name=weather.csv
model_id_name=weather
data_name=custom

seq_len=336

# Enable BOTH enhancements
multi_scale=1
channel_independent=0        # 0 = cross-channel (variables interact)

# Multi-scale configuration
patch_lengths="6,12,24"      # Three temporal scales
patch_strides="3,6,12"       # 50% overlap for each
patch_weights="0.2,0.5,0.3"  # Balanced fusion

for pred_len in 96 192 336 720
do
    python -u run_longExp.py \
      --random_seed $random_seed \
      --is_training 1 \
      --root_path $root_path_name \
      --data_path $data_path_name \
      --model_id $model_id_name'_multiCross_'$seq_len'_'$pred_len \
      --model $model_name \
 

In [16]:
!grep -R "PatchTST_backbone" -n PatchTST_supervised | head -n 30


PatchTST_supervised/models/PatchTST.py:7:from layers.PatchTST_backbone import PatchTST_backbone
PatchTST_supervised/models/PatchTST.py:67:            encoder = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:129:    - Run short group through single-scale PatchTST_backbone with short_patch_len/short_stride (model_short)
PatchTST_supervised/models/PatchTST.py:258:                    self.model_trend_short = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:282:                    self.model_res_short = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:324:                    self.model_short = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:345:                    self.model_trend = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:357:                    self.model_res = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:373:                    self.model_trend_long = PatchTST_backbone(
PatchTST_supervised/models/PatchTST.py:385:       

In [17]:
!sed -n '1,120p' PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


#!/bin/bash

# Multi-Scale + Cross-Channel PatchTST for Weather Forecasting
# Combines both enhancements for maximum performance:
#   - Multi-scale: Captures temporal patterns at 6hr, 12hr, 24hr scales
#   - Cross-channel: Enables interaction between weather variables

model_name=PatchTST
random_seed=2021

root_path_name=./dataset/
data_path_name=weather.csv
model_id_name=weather
data_name=custom

seq_len=336

# Enable BOTH enhancements
multi_scale=1
channel_independent=0        # 0 = cross-channel (variables interact)

# Multi-scale configuration
patch_lengths="6,12,24"      # Three temporal scales
patch_strides="3,6,12"       # 50% overlap for each
patch_weights="0.2,0.5,0.3"  # Balanced fusion

for pred_len in 96 192 336 720
do
    python -u run_longExp.py \
      --random_seed $random_seed \
      --is_training 1 \
      --root_path $root_path_name \
      --data_path $data_path_name \
      --model_id $model_id_name'_multiCross_'$seq_len'_'$pred_len \
      --model $model_name \
 

In [18]:
!grep -n "short_patch" -n PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh
!grep -n "short_idx" -n PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


47:      --short_patch_len 6 \      #Short Patch Config VV
49:      --short_idx "13,16" \      #Short Patch Config VV


In [19]:
!grep -n "short_stride" PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


48:      --short_stride 3 \         #Short Patch Config VV


In [20]:
import pandas as pd

df = pd.read_csv("/content/CSPatchTST/datasets/weather/weather.csv")
cols = df.columns.tolist()
print("Total columns:", len(cols))
print(cols)

print("raining index:", cols.index("raining (s)"))
print("max. wv index:", cols.index("max. wv (m/s)"))


Total columns: 22
['date', 'p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)', 'wd (deg)', 'rain (mm)', 'raining (s)', 'SWDR (W/m�)', 'PAR (�mol/m�/s)', 'max. PAR (�mol/m�/s)', 'Tlog (degC)', 'OT']
raining index: 16
max. wv index: 13


In [21]:
!grep -n "short_idx" PatchTST_supervised/scripts/PatchTST/weather_multiscale_crosschannel.sh


49:      --short_idx "13,16" \      #Short Patch Config VV


In [22]:
%cd /content/CSPatchTST/PatchTST_supervised
!ls


/content/CSPatchTST/PatchTST_supervised
data_provider  Formers	models		  run_longExp.py  utils
exp	       layers	requirements.txt  scripts


In [23]:
import torch
from types import SimpleNamespace
from models.PatchTST import Model

configs = SimpleNamespace(
    enc_in=21,
    seq_len=336,
    pred_len=96,

    e_layers=3,
    n_heads=16,
    d_model=128,
    d_ff=256,
    dropout=0.2,
    fc_dropout=0.2,
    head_dropout=0,
    individual=False,

    patch_len=16,
    stride=8,
    padding_patch=None,

    revin=True,
    affine=True,
    subtract_last=False,

    decomposition=False,
    kernel_size=25,

    channel_independent=0,
    multi_scale=1,
    patch_lengths=[6,12,24],
    patch_strides=[3,6,12],
    patch_weights=[0.2,0.5,0.3],

    short_patch_len=6,
    short_stride=3,
    short_idx=[13,16],
)

m = Model(configs)
x = torch.randn(2, configs.seq_len, configs.enc_in)
y = m(x)

print("y shape:", y.shape)


y shape: torch.Size([2, 96, 21])


In [24]:
%cd /content/CSPatchTST/PatchTST_supervised
!bash scripts/PatchTST/weather_multiscale_crosschannel.sh


/content/CSPatchTST/PatchTST_supervised
Multi-scale patching enabled:
  Patch lengths: [6, 12, 24]
  Strides: [3, 6, 12]
  Normalized weights: ['0.200', '0.500', '0.300']
Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='weather_multiCross_336_96', model='PatchTST', data='custom', root_path='./dataset/', data_path='weather.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', seq_len=336, label_len=48, pred_len=96, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, channel_independent=0, multi_scale=1, patch_lengths=[6, 12, 24], patch_strides=[3, 6, 12], patch_weights=[0.2, 0.5, 0.3], short_patch_len=6, short_stride=3, short_idx='', embed_type=0, enc_in=21, dec_in=7, c_out=7, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=1, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention

In [25]:
!grep -n "short_patch_len" -n run_longExp.py
!grep -n "short_stride" -n run_longExp.py
!grep -n "short_idx" -n run_longExp.py


62:    parser.add_argument('--short_patch_len', type=int, default=6)
63:    parser.add_argument('--short_stride', type=int, default=3)
64:    parser.add_argument('--short_idx', type=str, default="")


In [26]:
mkdir -p logs/LongForecasting


In [27]:
!bash -n scripts/PatchTST/weather_multiscale_crosschannel.sh


In [28]:
!sed -n '1,40p' logs/LongForecasting/PatchTST_MultiCross_weather_336_96.log


sed: can't read logs/LongForecasting/PatchTST_MultiCross_weather_336_96.log: No such file or directory


In [29]:
!ls -lh logs/LongForecasting | tail -n 20
!tail -n 40 logs/LongForecasting/PatchTST_MultiCross_weather_336_96.log


total 0
tail: cannot open 'logs/LongForecasting/PatchTST_MultiCross_weather_336_96.log' for reading: No such file or directory


In [30]:
%cd /content/CSPatchTST/PatchTST_supervised
!mkdir -p dataset
!cp /content/CSPatchTST/datasets/weather/weather.csv dataset/weather.csv
!ls -lh dataset



/content/CSPatchTST/PatchTST_supervised
total 6.9M
-rw-r--r-- 1 root root 6.9M Jan 26 04:11 weather.csv


In [31]:
%cd /content/CSPatchTST/PatchTST_supervised
!grep -n "np\.Inf" -n utils/tools.py


/content/CSPatchTST/PatchTST_supervised
47:        self.val_loss_min = np.Inf


In [33]:
!sed -i 's/np\.Inf/np.inf/g' utils/tools.py


In [32]:
!grep -n "np\.Inf" -n utils/tools.py || echo "np.Inf fixed"


47:        self.val_loss_min = np.Inf


In [36]:
!python -u run_longExp.py \
  --random_seed 2021 \
  --is_training 1 \
  --root_path ./dataset/ \
  --data_path weather.csv \
  --model_id weather_multiCross_336_96 \
  --model PatchTST \
  --data custom \
  --features M \
  --seq_len 336 \
  --pred_len 96 \
  --enc_in 21 \
  --channel_independent 0 \
  --multi_scale 1 \
  --patch_lengths "6,12,24" \
  --patch_strides "3,6,12" \
  --patch_weights "0.2,0.5,0.3" \
  --e_layers 3 \
  --n_heads 16 \
  --d_model 128 \
  --d_ff 256 \
  --dropout 0.2 \
  --fc_dropout 0.2 \
  --head_dropout 0 \
  --patch_len 16 \
  --stride 8 \
  --short_patch_len 6 \
  --short_stride 3 \
  --short_idx "13,16" \
  --des "MultiScale_CrossChannel_Exp" \
  --train_epochs 5 \
  --patience 3 \
  --itr 1 \
  --batch_size 16 \
  --learning_rate 0.0001


Multi-scale patching enabled:
  Patch lengths: [6, 12, 24]
  Strides: [3, 6, 12]
  Normalized weights: ['0.200', '0.500', '0.300']
Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='weather_multiCross_336_96', model='PatchTST', data='custom', root_path='./dataset/', data_path='weather.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', seq_len=336, label_len=48, pred_len=96, fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, channel_independent=0, multi_scale=1, patch_lengths=[6, 12, 24], patch_strides=[3, 6, 12], patch_weights=[0.2, 0.5, 0.3], short_patch_len=6, short_stride=3, short_idx='13,16', embed_type=0, enc_in=21, dec_in=7, c_out=7, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers